In [17]:
using LinearAlgebra
using Plots
using Primes

┌ Info: Precompiling Primes [27ebfcd6-29c5-5fa9-bf4b-fb8fc14df3ae]
└ @ Base loading.jl:1260


# Orthomads
### Overview
1. Quasi-random halton sequence produces a vector in $[0,1]^n$
2. Vector is scaled and rounded to an appropriate length, giving the *adjusted halton direction*
3. The housholder transformation is applied to the adjusted halton direction, giving $n$ orthononal and integer vectors, forming a basis for $\mathbb{R}^n$
4. The basis is completed to a positive basis formed by $2n$ orthomads poll directions $D_k$, by including in $D_k$ the basis and its negatives.

### Halton Sequence $u_t$
A deterministic family of sequences that grow dense in the hypercube $[0,1]^n$. The simplest such sequence has $t$th element
\begin{equation}
u_t = (u_{t,p_1},u_{t,p_2},\cdots,u_{t,p_n})^T \in [0,1]^n,
\end{equation}
where $p_j$ is the $j$th prime number. $u_{t,p}$ is the radical-inverse function in base $p$,
\begin{equation}
u_{t,p} = \sum^\infty_{r=0}\frac{a_{t,r,p}}{p^{1+r}},
\end{equation}
where $a_{t,r,p}\in \mathbb{Z}_+$ are the unique coefficients of the base $p$ expansion of $t$:
\begin{equation}
t = \sum^\infty_{r=0}a_{t,r,p}p^r,
\end{equation}

In [13]:
#The following 8 matrices correspond to the first 4 elements of uₜ for n=4 (ie, t=0...7)
u0 = [0, 0, 0, 0]
u1 = [1/2, 1/3, 1/5, 1/7]
u2 = [1/4, 2/3, 2/5, 2/7]
u3 = [3/4, 1/9, 3/5, 3/7]
u4 = [1/8, 4/9, 4/5, 4/7]
u5 = [5/8, 7/9, 1/25, 5/7]
u6 = [3/8, 2/9, 6/25, 6/7]
u7 = [7/8, 5/9, 11/25, 1/49]

4-element Array{Float64,1}:
 0.875
 0.5555555555555556
 0.44
 0.02040816326530612

In [92]:
#Return the corresponding sequence uₜ
function Halton(n, t)
    p = map(prime, 1:n)
    return map(p -> HaltonEntry(p,t), p)
end

function HaltonEntry(p,t)
    u = 0
    a_r = HaltonCoefficient(p,t)
    for (r,a) in enumerate(a_r)
        u += a/(p^r) #note that the equation is a/p^r+1, but julia indexes from 1
    end
    return u
end

function HaltonCoefficient(p,t)
    #Maximum non-zero value of r
    r_max = floor(Int64, log(p, t))
    #Need to give values for 0:r_max
    a = zeros(Int(r_max+1))
    t_local = t
    for r in r_max:-1:0
        t_local == 0 && break
        a[r+1] = floor(t_local/p^r)
        t_local -= p^r * a[r+1]
    end
    return a   
end

HaltonCoefficient (generic function with 1 method)

In [93]:
#Approx is only necessary for a few cases of floating point inaccuracy
@show Halton(4, 1) ≈ u1
@show Halton(4, 2) ≈ u2
@show Halton(4, 3) ≈ u3
@show Halton(4, 4) ≈ u4
@show Halton(4, 5) ≈ u5
@show Halton(4, 6) ≈ u6
@show Halton(4, 7) ≈ u7

Halton(4, 1) ≈ u1 = true
Halton(4, 2) ≈ u2 = true
Halton(4, 3) ≈ u3 = true
Halton(4, 4) ≈ u4 = true
Halton(4, 5) ≈ u5 = true
Halton(4, 6) ≈ u6 = true
Halton(4, 7) ≈ u7 = true


true

Can observe some correlation in the last entry for the smaller values of t. This can be removed by excluding the initial points, the paper starts at the $n$th prime number ($p_n$) denoted $t_0$, and named the *Halton seed*.

Two properties are given:
- $2u_t - e = 0 \Leftrightarrow n = t =1$
- $|2u_{t,p_i}-1| = |2u_{t,p_j}-1| \Leftrightarrow t=0$

In addition the following proposition is made:
The Halton sequence $\{u_t\}^\infty_{t=1}$ is dense in the hypercube $[0,1]^n$.

This implies that the union of all the directs in the sequence is dense in $[0,1]^n$. That is to say that any direction $v\in[0,1]^n$ is an accumulation point of the sequence $\{u_t\}^\infty_{t=1}$.

# Adjusted Halton direction $q_{t,l}$
$u_t$ does not satisfy the requirements for the directions in $D_k$. $u_t$ must be scaled, translated, and rounded. The operations depend on the mesh index $l_k$. This  is used to create the *adjusted halton direction* $q_{t,l}\in\mathbb{Z}^n$, a direction which has norm close to $2^{|l|/2}$. The normalised direction $\frac{q_{t,l}}{\|q_{t,l}\|}$ is also constructed to be close to $\frac{2u_t-e}{\|2u_t-e\|}$. The first property stated shows that $2u_t-e=0$ iif $t=n=1$, which is never the case in this algorithm (as it is started at the $n$th prime number, giving t strictly larger than 1).

Define the following sequence of functions based on the $t$th Halton direction $u_t$:
\begin{equation}
q_t(\alpha) = \text{round}\big(\alpha\frac{2u_t-e}{\|2u_t-e\|}\big) \in \mathbb{Z}^n \cap \big[-\alpha-\frac{1}{2},+\alpha+\frac{1}{2}\big]
\end{equation}
Round defines the operation that sets $-0.5=-1$ and $0.5=1$. The scalar $\alpha\in\mathbb{R}_+$ is a scaling factor selected s.t. $\|q_t(\alpha_{t,l})\|$  is a close as possible to $2^{|l|/2}$ without exceeding it:
\begin{equation}
\alpha_{t,l} \in argmax_{\alpha\in\mathbb{R}_+}\|q_t(\alpha)\| \\
\text{s.t.  } \|q_t(\alpha)\|\le 2^{|l|/2}
\end{equation}

In [228]:
function AdjustedHaltonFamily(halt)
    d = 2 * halt .- 1
    q(α) = round.(α .* d ./ norm(d))
    return q
end

function AdjustedHalton(halt, n, l)
    #A function that describes a family of directions
    q = AdjustedHaltonFamily(halt)
    #Need to find the argument, α, of q that maximises l2 norm of result, s.t. it is ≤2^(|l|/2)
    #∃ optimal solution satisfying α ≥ 2^(|l|/2)/√n -0.5, ∴ use as starting point
    α = (2^(abs(l)/2)/sqrt(n)) - 0.5
    
    α = bad_argmax(α, x -> norm(q(x)), 2^(abs(l)/2))
    
    return q(α)
end

#should find some numerical line search for this
function bad_argmax(x, f, lim; iter_lim = 10)
    bump = 1
    iter = 1
    
    while iter < iter_lim
        t = x + bump
        if lim >= f(t)
            x = t
        else
            bump /= 2
        end
        iter += 1
    end
    return x
end

bad_argmax (generic function with 1 method)

In [190]:
n = 4 
t = 7
l = 0

h = Halton(n, t)
#Results match paper for all their examples, however the α selection is poor and does not give the same results
#However the rounding seems to make up for this
AdjustedHalton(h, n, l)

([0.0, 0.0, -0.0, -1.0], 0.8125)

# Orthogonal integer basis construction

Given an integer nonzero vector $q \in \mathbb{Z}^n$, the symmetricscaled householder transformation is used to construct an orthogonal basis for $\mathbb{R}^n$ composed of integer vectors:
\begin{equation}
H=\|q\|^2(I_n-2vv^T), \text{where } v =\frac{q}{\|q\|}.
\end{equation}
Dividing the previous equation by $\|q\|^4$ and applying symmetry, the inverse of H is given as $H^{-1}=\frac{H}{\|q\|^4}$. 

As $\exists H^{-1}$ the cols of $H$ form a basis in $\mathbb{R}^n$. The entries of $H$ are integer as $q$ and $\|q\|^2$ are integer.

In [229]:
function HouseHolder(q)
    nq = norm(q)
    v = q./nq
    return nq^2 .* (I - 2*v*v')
end

HouseHolder (generic function with 1 method)

In [240]:
n = 2
t = 6
l = 3

function get_dirs(n, t, l)
    h = Halton(n, t)
    #Results match paper for all their examples, however the α selection is poor and does not give the same results
    #However the rounding seems to make up for this
    q = AdjustedHalton(h, n, l)
    
    H = HouseHolder(q)
    
    D = hcat(H, -H)
    return D
end
f(n, t, l) = @time get_dirs(n, t, l);

In [264]:
f(2, 6, 3)

Dₖ = get_dirs(4, 13, 6)


#scatter(Dₖ[1,:],Dₖ[2,:], legend=false)

  0.000022 seconds (28 allocations: 2.391 KiB)


4×8 Array{Float64,2}:
  36.0   0.0  -18.0  -36.0  -36.0   -0.0   18.0  36.0
   0.0  54.0    0.0    0.0   -0.0  -54.0   -0.0  -0.0
 -18.0   0.0   36.0  -36.0   18.0   -0.0  -36.0  36.0
 -36.0   0.0  -36.0  -18.0   36.0   -0.0   36.0  18.0

# OrthoMads 

0. Initialisation

    $x_0\in \Omega, l_0 = 0, k=0, t_0=p_n$
    
1. Iteration k
    - Search
        
        Evaluate $f$ on a finite set $S_k \subset M_k$
    - Poll
    
        if the poll size is the smallest so far: $t_k = l_k + t_0$
        
        else: $t_k = 1 + \max\{t_j:j=0,1,\cdots,k-1\}$
        
        Then compute $u_{t_k}$, $q_{t_k,l_k}$, $H_{t_k,l_k}$, and $D_k=[H_{t_k,l_k},H_{t_k,l_k}]$
        
        Finally evaluate $f$ on $P_k \subset M_k$
2. Updates

    If the iteration is successful then: $x_{k+1} = x_s \text{ or } x_p$, and$l_{k+1} = l_k -1$

    Otherwise: $x_{k+1} = x_k$, and $l_{k+1} = l_k +1$.
    
    Finally: $k = k + 1$, and goto step $1$ if no stopping condition is met.